# 🖼️ Generate Screen Descriptions
This notebook runs a selected Vision-Language Model (VLM) on 100 annotated Dutch desktop screenshots using a chosen prompt, generating structured screen descriptions for blind users.

# 🛠️ Configuration

In [31]:
MODEL_NAME = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"
PROMPT_VERSION = "saliency"

PROMPT_TEXT = """
Beschrijf deze schermafbeelding voor een blinde gebruiker, met nadruk op de belangrijkste en functioneel relevante onderdelen.

Begin met het type scherm en de applicatie of website (indien zichtbaar). Beschrijf vervolgens het hoofddoel van het scherm en de belangrijkste elementen die een gebruiker nodig heeft om ermee te werken: koppen, knoppen, formulieren, foutmeldingen of statusinformatie.

Vermijd irrelevante of decoratieve details. Noem alleen elementen die bijdragen aan begrip of interactie met het scherm. Houd de beschrijving compact, neutraal en volledig in het Nederlands.
"""


# 📦 Step 1: Setup and Imports

In [32]:
import os
import base64
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
from together import Together
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr, spearmanr

load_dotenv()
client = Together()

# 📂 Step 2: Load Annotations

In [33]:
with open("annotation_dataset/annotations.json", "r", encoding="utf-8") as f:
    annotations = json.load(f)

# 🧠 Step 3: Define Encoding and Query Functions

In [34]:
def encode_image_to_data_url(image_path):
    """
    Convert a local PNG file to a base64-encoded data URL.
    This format is required by the Together API for vision inputs.
    """
    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:image/png;base64,{b64}"

def query_model(image_data_url, prompt):
    """
    Send a prompt and encoded image to the selected model on Together.
    Returns the model-generated screen description.
    """
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": image_data_url}}
            ]
        }]
    )
    return response.choices[0].message.content.strip()

# 📸 Step 4: Evaluate One Image

In [35]:
def evaluate_image_entry(entry, prompt, prompt_version):
    """
    Processes one image entry:
    - Loads the image by ID
    - Encodes it for Together API
    - Sends it with the prompt
    - Returns a structured result dict
    """
    image_id = entry["image_id"]
    image_path = Path(f"annotation_dataset/images/{image_id}.png")

    if not image_path.exists():
        print(f"⚠️ Skipping {image_id} — image not found at {image_path}")
        return None

    print(f"🔍 Analyzing image {image_id}...")

    data_url = encode_image_to_data_url(image_path)
    model_output = query_model(data_url, prompt)

    print(f"✅ Done: {image_id}")
    print(f"📝 Output: {model_output.strip()}\n")

    return {
        "image_id": image_id,
        "prompt_version": prompt_version,
        "prompt": prompt,
        "model": MODEL_NAME,
        "model_output": model_output.strip()
    }

# 🚀 Step 5: Run Inference on All Screens

In [ ]:
results = []
for i, entry in enumerate(annotations):
    result = evaluate_image_entry(entry, PROMPT_TEXT, PROMPT_VERSION)
    if result:
        results.append(result)

df_results = pd.DataFrame(results)


🔍 Analyzing image 001...
✅ Done: 001
📝 Output: Dit is een webpagina van het Algoritmeregister van de Nederlandse overheid, weergegeven in een browservenster. De pagina heeft als hoofddoel om informatie te bieden over algoritmes die door Nederlandse overheidsorganisaties worden gebruikt.

De belangrijkste elementen op de pagina zijn:

- Een navigatiebalk met de opties "Home", "Algoritmes", "Organisaties" en "Dashboard".
- Een zoekveld met de tekst "Vind een van de 822 algoritmes" en een knop "Zoeken".
- Een sectie met de drie meest recent gewijzigde algoritmebeschrijvingen, inclusief links naar deze algoritmes.
- Een link om door alle algoritmes te bladeren.
- Een informatieblok "Over dit register" dat uitlegt wat het Algoritmeregister is en hoe algoritmes geregistreerd kunnen worden.
- Drie secties onderaan de pagina: "Over algoritmes", "Samen doorontwikkelen" en "Wat kan ik hier doen?", die elk meer informatie bieden over respectievelijk wat algoritmes zijn, hoe men kan bijdragen aan 

# 💾 Step 6: Save Results to JSON

In [ ]:
import os
from datetime import datetime

results_dir = "descriptions"
os.makedirs(results_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
file_basename = f"{MODEL_NAME.replace('/', '_')}__{PROMPT_VERSION}__{timestamp}"
filepath = os.path.join(results_dir, f"{file_basename}.xlsx")

df_results.to_excel(filepath, index=False)
print(f"✅ Descriptions saved to {filepath}")

✅ Results saved to results\meta-llama_Llama-4-Maverick-17B-128E-Instruct-FP8__saliency__20250629_1115.xlsx
